In [3]:
# unzips the file into custom_data
import zipfile
import os

zip_path = r'C:\Users\Marcus\Documents\braintumor\my_model\data.zip'
extract_path = r'C:\Users\Marcus\Documents\braintumor\my_model\custom_data'


# Create the output directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzip complete!")


✅ Unzip complete!


In [5]:
#algorithm for transforming the custom_data to train and validation
!curl -o train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
!python train_val_split.py --datapath="C:/Users/Marcus/Documents/braintumor/my_model/custom_data" --train_pct=0.9


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3203  100  3203    0     0   7856      0 --:--:-- --:--:-- --:--:--  7869
100  3203  100  3203    0     0   7854      0 --:--:-- --:--:-- --:--:--  7869


Created folder at C:\Users\Marcus\Documents\braintumor\my_model\data/train/images.
Created folder at C:\Users\Marcus\Documents\braintumor\my_model\data/train/labels.
Created folder at C:\Users\Marcus\Documents\braintumor\my_model\data/validation/images.
Created folder at C:\Users\Marcus\Documents\braintumor\my_model\data/validation/labels.
Number of image files: 253
Number of annotation files: 253
Images moving to train: 227
Images moving to validation: 26


In [ ]:
pip install ultralytics

In [11]:
import ultralytics
print(ultralytics.__version__)

#make sure that you have it correctly installed

8.3.103


In [25]:
#training configurations

# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  # replace with your path here 
  data = {
      'path': 'C:/Users/Marcus/Documents/braintumor/my_model/custom_data',
      'train': 'C:/Users/Marcus/Documents/braintumor/my_model/data/train',
      'val': 'C:/Users/Marcus/Documents/braintumor/my_model/data/validation',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = 'C:/Users/Marcus/Documents/braintumor/my_model/custom_data/classes.txt'
path_to_data_yaml = 'C:/Users/Marcus/Documents/braintumor/my_model/custom_data/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
with open(path_to_data_yaml, 'r') as f:
    print(f.read())

#it should return your classes/names

Created config file at C:/Users/Marcus/Documents/braintumor/my_model/custom_data/data.yaml

File contents:

path: C:/Users/Marcus/Documents/braintumor/my_model/custom_data
train: C:/Users/Marcus/Documents/braintumor/my_model/data/train
val: C:/Users/Marcus/Documents/braintumor/my_model/data/validation
nc: 2
names:
- Brain Tumor
- Healthy Brain



In [ ]:
import subprocess

# Path to the data.yaml file
data_path = 'C:/Users/Marcus/Documents/braintumor/my_model/custom_data/data.yaml'

# Command for training with options to adjust batch size, workers, and log output
command = f"yolo detect train data={data_path} epochs=60 imgsz=640 batch=8 workers=4"

# Run the command and capture output in real-time
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8')

# Print the output while the process runs
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None:
        break
    if output:
        print(output.strip())  # Print the output line

# Capture and print errors if there are any
stderr_output = process.stderr.read()
if stderr_output:
    print(f"ERROR: {stderr_output}")

# Wait for the process to finish
process.wait()

print("Training completed.")


#or run it in terminal if youre having errors
# !yolo detect train data=C:/Users/Marcus/Documents/braintumor/my_model/custom_data/data.yaml model=yolo11s.pt epochs=60 imgsz=640


Ultralytics 8.3.103 🚀 Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 5 5500)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/Users/Marcus/Documents/braintumor/my_model/custom_data/data.yaml, epochs=60, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labe

In [1]:
#if The model has been trained; now it's time to test it! The commands below run the model on the images in the validation folder and then display
#the results for the first 10 images. This is a good way to confirm your model is working as expected. Click Play on the blocks below to see how your
#model performs.

!yolo detect predict model=C:/Users/Marcus/Documents/braintumor/my_model/runs/detect/train/weights/best.pt source=C:/Users/Marcus/Documents/braintumor/my_model/data/validation/images save=True

In [ ]:
#save the prediction result

import glob
from IPython.display import Image, display
for image_path in glob.glob(f'C:/Users/Marcus/Documents/braintumor/my_model/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')
